<a href="https://colab.research.google.com/github/u-masao/vector-search-tutorial-for-japanese/blob/main/Session03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語のためのベクトル検索ハンズオン：Session 3

このセッションでは Qdrant 公式サイトのクイックスタートを実行します。

Docker で Qdrant を実行して、Python SDK で基本的な操作をします。

## 環境構築


### Google Colab 向け Docker インストール

In [1]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


In [2]:
# udocker("pull qdrant/qdrant")

In [3]:
udocker("run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant")

# 初回実行時は正常に終了しないことがあります。しばらくして応答がなければランタイムをリセットしてください。

user@pc$ udocker run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 6bad72c1-be3d-3bce-9d0c-b64ebdd80def                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: entrypoint.sh
           _                 _    
  __ _  __| |_ __ __ _ _ __ | |_  
 / _` |/ _` | '__/ _` | '_ \| __| 
| (_| | (_| | | | (_| | | | | |_  
 \__, |\__,_|_|  \__,_|_| |_|\__| 
    |_|                           

Version: 1.8.4, build: 984f55d6
Access web UI at http://localhost:6333/dashboard

2024-04-21T12:43:20.026769Z  INFO storage::content_manager::consensus::persistent: Initializing new raft state at ./storage/raft_state.json    
2024-04-21T12:43:20.399566Z  INFO qdrant: Distrib

###  SageMaker 向け docker インストール

(省略)

## パッケージのインストールと読み込み

In [4]:
!pip install -q qdrant-client japanize_matplotlib sentence-transformers

In [5]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

## Qdrant の利用

以下を実行します。

- Qdrant Client を初期化
- コレクションを初期化(内積計算モード)
- ポイント(アイテム)を登録
- ベクトル検索のクエリ例
- ベクトル検索&フィルタリングのクエリ例

In [6]:
from qdrant_client import QdrantClient

qdrant_url = "http://localhost:6333"
qdrant_collection_name = "test_collection"

# Qdrant Client を初期化
client = QdrantClient(url=qdrant_url)

In [ ]:
# コレクションの情報を取得
# status = client.get_collection( collection_name=qdrant_collection_name )
# print(status)

# コレクションを削除
# status = client.delete_collection( collection_name=qdrant_collection_name )
# print(status)

In [8]:
from qdrant_client.models import Distance, VectorParams

# コレクション(テーブル)を初期化
# ベクトル次元 4次元、距離: 内積
# (既にコレクションが定義されている場合はエラーが出ます)
#  -> b'{"status":{"error":"Wrong input: Collection `test_collection` already exists!"},"time":0.0000995}'
client.create_collection(
    collection_name=qdrant_collection_name,
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [9]:
from qdrant_client.models import PointStruct

# ポイント(アイテム)を追加
# ベクトルとペイロード( {"city":"xxxx"} )
operation_info = client.upsert(
    collection_name=qdrant_collection_name,
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [10]:
# ベクトル検索
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=[0.2, 0.1, 0.9, 0.7],  # 検索キーベクトルを指定
    limit=3
)

_ = [print(x) for x in search_result]

id=4 version=0 score=1.362 payload={'city': 'New York'} vector=None shard_key=None
id=1 version=0 score=1.273 payload={'city': 'Berlin'} vector=None shard_key=None
id=3 version=0 score=1.208 payload={'city': 'Moscow'} vector=None shard_key=None


In [11]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

# ベクトル検索 & フィルタリングの検索例
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=[0.2, 0.1, 0.9, 0.7],  # ベクトル検索キー
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]  # ペイロードでフィルタリング
    ),
    with_payload=True,  # ペイロードを取得
    limit=3,
)

_ = [print(x) for x in search_result]

id=2 version=0 score=0.871 payload={'city': 'London'} vector=None shard_key=None
